# Part 3 - Evaluate simulations with experimental analogue

### First load some references

In [ ]:
#r "BoSSSpad.dll"
// #r "..\..\src\L4-application\BoSSSpad\bin\Release\net5.0\BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases.PrintingNip;
Init();

#### Check if Output should be generated

Only if the output directories were previously created via `Part0_PrintingNip_Setup` output is stored to the subdirectories.  
Otherwise, datatables are stored directly to the working directory and plots displayed in the notebook.  
The plots are not adjusted to look "nice" in the notebook.  
This is not part of the `ValidationTestRunner`.

In [ ]:
static bool GenerateOutput = Directory.Exists(@"./PrintingNip") && Directory.Exists(@"./PrintingNip/Figures") && Directory.Exists(@"./PrintingNip/Files") && Directory.Exists(@"./PrintingNip/Output");

### Init Database etc.

In [ ]:
string ProjectName = "PrintingNip_Part3";

In [ ]:
BoSSSshell.WorkflowMgm.Init(ProjectName);

In [ ]:
static var myDb = BoSSSshell.WorkflowMgm.DefaultDatabase;

### Load Simulations

In [ ]:
var sessions = myDb.Sessions.Where(s => s.ProjectName == ProjectName && s.SuccessfulTermination).ToArray();

In [ ]:
sessions.Count()

### Setup Post-Processing

In [ ]:
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Clear();

In [ ]:
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("PressureRange", Postprocessing.PressureRange);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("VelocityXMax", Postprocessing.VelocityXMax);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("Massflux", Postprocessing.Massflux);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("NipShearRate", Postprocessing.NipShearRate);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("NipShearRateCylinder", Postprocessing.NipShearRateCylinder);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("ShearStressCylinder", Postprocessing.ShearStressCylinder);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("NipShearRateSubstrate", Postprocessing.NipShearRateSubstrate);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("ShearStressSubstrate", Postprocessing.ShearStressSubstrate);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("ViscousDissipation", Postprocessing.ViscousDissipation); 
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("PositionOfSynchronousFlow", Postprocessing.PositionOfSynchronousFlow);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("PositionOfStagnatingFlow", Postprocessing.PositionOfStagnatingFlow);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("dPdXatSynchronousPoint", Postprocessing.dPdXatSynchronousPoint);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("dPdXatStagnationPoint", Postprocessing.dPdXatStagnationPoint);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("dPdXatNip", Postprocessing.dPdXatNip);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("dPdXatConstantX", Postprocessing.dPdXatConstantX);

### Create Datatable

In [ ]:
DataTable tab;
if(GenerateOutput){
    if(!File.Exists("./PrintingNip/Files/PrintingNip-Part3Export.csv")){
        tab = sessions.GetSessionTable(BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Select(kv => new Tuple<string, Func<ISessionInfo, object>>(kv.Key, kv.Value)).ToArray());
        tab.ToCSVFile("./PrintingNip/Files/PrintingNip-Part3Export.csv", ColSep: '%');
    } else{
        tab = TableExtensions.FromCSVFile("./PrintingNip/Files/PrintingNip-Part3Export.csv", ColSep: '%');
    }
} else {
    // in this case always recreate the datatable
    tab = sessions.GetSessionTable(BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Select(kv => new Tuple<string, Func<ISessionInfo, object>>(kv.Key, kv.Value)).ToArray());
    tab.ToCSVFile("./PrintingNip-Part3Export.csv", ColSep: '%');     
}

### Export Datatable

In [ ]:
string[] Columns = new string[] {"PressureRange", "VelocityXMax", "Massflux", "NipShearRate", "NipShearRateCylinder", "ShearStressCylinder", "NipShearRateSubstrate", "ShearStressSubstrate", "ViscousDissipation", "PositionOfSynchronousFlow",  "PositionOfStagnatingFlow", "dPdXatSynchronousPoint", "dPdXatStagnationPoint", "dPdXatNip", "dPdXatConstantX"};
string[] Labels = new string[] {"Pressure", "Velocity", "Massflux", "Shearrate (Nip)", "Shearrate (Cylinder)", "Shearforce (Cylinder)", "Shearrate (Substrate)", "Shearforce (Substrate)", "Dissipation", "$x_{sync}$", "$x_{stag}$", @"$\\\frac{\\\partial p}{\\\partial x}_{sync}$", @"$\\\frac{\\\partial p}{\\\partial x}_{stag}$", @"$\\\frac{\\\partial p}{\\\partial x}_{nip}$", @"$\\\frac{\\\partial p}{\\\partial x}$"};
string[] Units = new string[] {@"$\\\left[\\\frac{N}{m^2}\\right]$", @"$\\\left[\\\frac{m}{s}\\right]$", @"$\\\left[\\\frac{kg}{s}\\right]$", @"$\\\left[\\\frac{1}{s}\\right]$", @"$\\\left[\\\frac{1}{s}\\right]$", @"$\\\left[N\\right]$", @"$\\\left[\\\frac{1}{s}\\right]$", @"$\\\left[N\\right]$", @"$\\\left[W\\right]$", @"$\\\left[m\\right]$", @"$\\\left[m\\right]$", @"$\\\left[\\\frac{N}{m^3}\\right]$", @"$\\\left[\\\frac{N}{m^3}\\right]$", @"$\\\left[\\\frac{N}{m^3}\\right]$", @"$\\\left[\\\frac{N}{m^3}\\right]$"}; // some escape character nonsense

In [ ]:
var tab2export = tab.ExtractColumns(Columns.ToList().Prepend("id:V_Wall").Prepend("id:P_Diff").Prepend("id:Radius").Prepend("id:delta").Prepend("id:Raster").Prepend("SessionName").ToArray());
if(GenerateOutput){
    tab2export.ToHTMLFile("PrintingNip-Part3.html", Path.GetFullPath("./PrintingNip/Files"));
    tab2export.ToCSVFile("./PrintingNip/Files/PrintingNip-Part3.csv", ColSep: ';');
} else {
    tab2export.ToHTMLFile("PrintingNip-Part3.html", Path.GetFullPath("./"));
    tab2export.ToCSVFile("./PrintingNip-Part3.csv", ColSep: ';');
}

### Plot Correlation - Constant Nip Width

Plot all Properties, we looked at before as functions of nip width. Remember, this time around the pressure b.c. is such that the stagnation point should remain constant

In [ ]:
var tab2plt = tab.ExtractRows((i,row) => Convert.ToDouble(row["id:P_Diff"]) > 0.0 && Convert.ToDouble(row["id:delta"]).ApproxEqual(1e-5)); // arbitrary nip width

In [ ]:
List<Plot2Ddata> plts = new List<Plot2Ddata>();
foreach(string col in Columns){
    int k = Columns.IndexOf(col);
    var plt = tab2plt.ToPlot("id:V_Wall", col, ColName_GroupSelection: new string[] { "id:Raster" });    
    plt.LogX   = true;
    plt.LogY   = true;
    plt.Xlabel = @"Printing Velocity Width $\\\left[\\\frac{m}{s}\\right]$";
    plt.Ylabel = Labels[k] + " " + Units[k];
    plt.LabelTitleFont = 32;
    plt.LabelFont = 24;
    plt.LegendFont = 18;
    plt.lmargin = 12;
    plt.bmargin = 4;
    plt.ShowLegend = true;    
    plt.LegendBox = true;
    if(plt.Regression().Average(kv => kv.Value) > 0){
        plt.LegendAlignment = new string[]{"i", "l", "t"};
        plt.LegendSwap = true;    
    } else{
        plt.LegendAlignment = new string[]{"i", "r", "t"};
        plt.LegendSwap = false;    
    }
    for(int n= 0; n < plt.dataGroups.Count(); n++){
        plt.dataGroups[n].Format.LineWidth = 2;
        plt.dataGroups[n].Format.PointSize = 1;
        plt.dataGroups[n].Name = "Raster $ = " + Convert.ToDouble(plt.dataGroups[n].Name.Split("id:Raster").Last()).ToString() + @" \\\frac{1}{cm}$";
    }
    plt.ModFormat();
    plts.Add(plt);
}

In [ ]:
if(GenerateOutput){
    using(StreamWriter stw = new StreamWriter("./PrintingNip/Output/SlopesOverRaster.txt")){
        foreach(var plt in plts){
            var slopes = plt.Regression();
            stw.WriteLine("<===================================================>");
            stw.WriteLine(plt.Ylabel + ":");
            slopes.ForEach(kvp => stw.WriteLine("\t" + kvp.Key + " : " + kvp.Value));
            stw.WriteLine("<===================================================>");
        }
    }
} else {
    {
        StringBuilder stw = new StringBuilder();
        foreach(var plt in plts){
            var slopes = plt.Regression();
            stw.AppendLine("<===================================================>");
            stw.AppendLine(plt.Ylabel + ":");
            slopes.ForEach(kvp => stw.AppendLine("\t" + kvp.Key + " : " + kvp.Value));
            stw.AppendLine("<===================================================>");
        }
        stw.ToString().Display();
    }
}

In [ ]:
int k = 0;
foreach(var plt in plts){
    var gp = new Gnuplot();
    gp.PlotLogSlope(plt, format: new PlotFormat(lineColor: (LineColors)7));
    gp.Cmd("set key box width -9");
    plt.ToGnuplot(gp);
    if(GenerateOutput){
        gp.PlotCairolatex().SaveTo("./PrintingNip/Figures/Part3/H0_"+Columns[k]+".tex");
    } else {
        gp.PlotNow().Display();
    }
    k++;
}

### Plot Correlation - Constant Raster

In [ ]:
var tab2plt = tab.ExtractRows((i,row) => Convert.ToDouble(row["id:P_Diff"]) > 0.0 && Convert.ToDouble(row["id:Raster"]).ApproxEqual(80)); // arbitrary nip width

In [ ]:
List<Plot2Ddata> plts = new List<Plot2Ddata>();
foreach(string col in Columns){
    int k = Columns.IndexOf(col);
    var plt = tab2plt.ToPlot("id:V_Wall", col, ColName_GroupSelection: new string[] { "id:delta" });    
    plt.LogX   = true;
    plt.LogY   = true;
    plt.Xlabel = @"Printing Velocity $\\\left[\\\frac{m}{s}\\right]$";
    plt.Ylabel = Labels[k] + " " + Units[k];
    plt.LabelTitleFont = 32;
    plt.LabelFont = 24;
    plt.LegendFont = 18;
    plt.lmargin = 12;
    plt.bmargin = 4;
    plt.ShowLegend = true;    
    plt.LegendBox = true;
    if(plt.Regression().Average(kv => kv.Value) > 0){
        plt.LegendAlignment = new string[]{"i", "l", "t"};
        plt.LegendSwap = true;    
    } else{
        plt.LegendAlignment = new string[]{"i", "r", "t"};
        plt.LegendSwap = false;    
    }     
    for(int n= 0; n < plt.dataGroups.Count(); n++){
        plt.dataGroups[n].Format.LineWidth = 2;
        plt.dataGroups[n].Format.PointSize = 1;
        plt.dataGroups[n].Name = @"$\\\varepsilon = " + Convert.ToDouble(plt.dataGroups[n].Name.Split("id:delta").Last()).ToString("0e0").Replace("e","\\\\times 10^{") + "}" + @" m$";
    }
    plt.ModFormat();
    plts.Add(plt);
}

In [ ]:
if(GenerateOutput){
    using(StreamWriter stw = new StreamWriter("./PrintingNip/Output/SlopesOverNipWidth.txt")){
        foreach(var plt in plts){
            var slopes = plt.Regression();
            stw.WriteLine("<===================================================>");
            stw.WriteLine(plt.Ylabel + ":");
            slopes.ForEach(kvp => stw.WriteLine("\t" + kvp.Key + " : " + kvp.Value));
            stw.WriteLine("<===================================================>");
        }
    }
} else {
    {
        StringBuilder stw = new StringBuilder();
        foreach(var plt in plts){
            var slopes = plt.Regression();
            stw.AppendLine("<===================================================>");
            stw.AppendLine(plt.Ylabel + ":");
            slopes.ForEach(kvp => stw.AppendLine("\t" + kvp.Key + " : " + kvp.Value));
            stw.AppendLine("<===================================================>");
        }
        stw.ToString().Display();
    }
}

In [ ]:
int k = 0;
foreach(var plt in plts){
    var gp = new Gnuplot();
    gp.PlotLogSlope(plt, format: new PlotFormat(lineColor: (LineColors)7));
    gp.Cmd("set key box width -5");
    plt.ToGnuplot(gp);
    if(GenerateOutput){
        gp.PlotCairolatex().SaveTo("./PrintingNip/Figures/Part3/R0_"+Columns[k]+".tex");
    } else {
        gp.PlotNow().Display();
    }
    k++;
}

### Plot Correlation - Constant Raster, Nip-Width on abscissa

In [ ]:
var tab2plt = tab.ExtractRows((i,row) => Convert.ToDouble(row["id:P_Diff"]) > 0.0 && Convert.ToDouble(row["id:Raster"]).ApproxEqual(80)); // arbitrary nip width

In [ ]:
List<Plot2Ddata> plts = new List<Plot2Ddata>();
foreach(string col in Columns){
    int k = Columns.IndexOf(col);
    var plt = tab2plt.ToPlot("id:delta", col, ColName_GroupSelection: new string[] { "id:V_Wall" });    
    plt.LogX   = true;
    plt.LogY   = true;
    plt.Xlabel = @"Nip Width $\\\left[m]$";
    plt.Ylabel = Labels[k] + " " + Units[k];
    plt.LabelTitleFont = 32;
    plt.LabelFont = 24;
    plt.LegendFont = 18;
    plt.lmargin = 12;
    plt.bmargin = 4;
    plt.ShowLegend = true;    
    plt.LegendBox = true;
    if(plt.Regression().Average(kv => kv.Value) > 0){
        plt.LegendAlignment = new string[]{"i", "l", "t"};
        plt.LegendSwap = true;    
    } else{
        plt.LegendAlignment = new string[]{"i", "r", "t"};
        plt.LegendSwap = false;    
    }     
    for(int n= 0; n < plt.dataGroups.Count(); n++){
        plt.dataGroups[n].Format.LineWidth = 2;
        plt.dataGroups[n].Format.PointSize = 1;
        plt.dataGroups[n].Name = @"$V_Wall = " + Convert.ToDouble(plt.dataGroups[n].Name.Split("id:V_Wall").Last()) + "}" + @" \\\frac{m}{s}$";
    }
    plt.ModFormat();
    plts.Add(plt);
}

In [ ]:
int k = 0;
foreach(var plt in plts){
    var gp = new Gnuplot();
    gp.PlotLogSlope(plt, format: new PlotFormat(lineColor: (LineColors)7));
    gp.Cmd("set key box width -5");
    plt.ToGnuplot(gp);
    if(GenerateOutput){
        gp.PlotCairolatex().SaveTo("./PrintingNip/Figures/Part3/R0_"+Columns[k]+"OverH.tex");
    } else {
        gp.PlotNow().Display();
    }
    k++;
}

### Comparison to experiment

In [ ]:
string[] lines = File.ReadAllLines("./ExperimentalResults.txt");
List<Tuple<double, double, double, double>> ExpResult = new List<Tuple<double, double, double, double>>(); // <Raster,V,dpdx,dpdx(std)>

double e = 0.0395; // viscosity
double s = 0.027; // surface tension

for(int i = 1; i < lines.Length; i++){
    string line = lines[i];
    string[] values = line.Split('\t');
    double raster = Convert.ToDouble(values[0]);
    double velocity = Convert.ToDouble(values[1]);
    double finger = Convert.ToDouble(values[2])*100; // convert to 1/m
    double fingerStd = Convert.ToDouble(values[3])*100; // standard deviation, convert to 1/m

    double dpdx = 16*s*finger*finger;
    double dpdxStd = 32*s*finger*fingerStd;
    
    ExpResult.Add(Tuple.Create(raster, velocity, dpdx, dpdxStd));
}

In [ ]:
double[] raster = new double[]{60.0, 70.0, 80.0, 100.0};
var Plots = new Gnuplot[raster.Length];
for(int i = 0; i < raster.Length; i++){

    // data for experiment
    var Exp = ExpResult.Where(r => r.Item1 == raster[i]);
    var xExp = Exp.Select(r => r.Item2);
    var yExp = Exp.Select(r => r.Item3);
    var yDevExp = Exp.Select(r => r.Item4);

    // data for simulation
    var tab2plt = tab.ExtractRows((j,row) => Convert.ToDouble(row["id:delta"]) == 1e-5 && Convert.ToDouble(row["id:Raster"]).ApproxEqual(raster[i])); // arbitrary nip width
    var xSim = tab2plt.Select().Select(row => Convert.ToDouble(row["id:V_Wall"])).ToArray();
    var ySim = tab2plt.Select().Select(row => Convert.ToDouble(row["dPdXatStagnationPoint"])).ToArray();
    Array.Sort(xSim, ySim);

    // Add Data to plot
    var gp = new Gnuplot();
    Plots[i] = gp;
    gp.PlotXY(xExp, yExp, "Experiment", new PlotFormat(null, LineColors.Red, DashTypes.Solid, 2, Styles.LinesPoints, PointTypes.Circle), logX: true, logY: true, useY2: false);
    gp.PlotXY(xExp, yExp.Select((x, j) => x + yDevExp.ElementAt(j)), null, new PlotFormat(null, LineColors.Red, DashTypes.Dashed, 1, Styles.LinesPoints, PointTypes.Plus), logX: true, logY: true, useY2: false);
    gp.PlotXY(xExp, yExp.Select((x, j) => x - yDevExp.ElementAt(j)), null, new PlotFormat(null, LineColors.Red, DashTypes.Dashed, 1, Styles.LinesPoints, PointTypes.Plus), logX: true, logY: true, useY2: false);

    //gp.SetYRange(0.9 * ySim.Min(), 1.1 * ySim.Max());
    gp.SetYRange(0.9 * Math.Min(ySim.Min(), yExp.Min()), 1.1 * Math.Max(ySim.Max(), yExp.Max()));
    gp.SetXRange(0.1, 1.5);
    //gp.SetY2Range(0.9 * yExp.Min(), 1.1 * yExp.Min() * ySim.Max() / ySim.Min());

    Plot2Ddata dat = new Plot2Ddata(xSim.Select(x => Math.Log10(x)), ySim.Select(x => Math.Log10(x)));
    double slope = Math.Round(dat.Regression().Average(kv => kv.Value) / 0.1) * 0.1;    
    gp.PlotXY(new double[] {xSim.Min(), xSim.Max()}, new double[] {ySim.Average() * Math.Pow((xSim.Min()/xSim.Average()), slope), ySim.Average() * Math.Pow((xSim.Max()/xSim.Average()), slope) }, null, new PlotFormat(null, LineColors.Black, DashTypes.Solid, 1), logX: true, logY: true);

    using (StringWriter stw = new StringWriter()) {
        double[,] points = new double[4, 2];
        points[0, 0] = xSim.Average();
        points[1, 0] = 1.1 * xSim.Average();
        points[2, 0] = 1.1 * xSim.Average();
        points[3, 0] = xSim.Average();

        points[0, 1] = ySim.Average();
        points[1, 1] = ySim.Average();
        points[2, 1] = ySim.Average() * Math.Pow(1.1, slope);
        points[3, 1] = ySim.Average() * Math.Pow(1.1, slope);

        stw.Write("set object poly from ");
        
        stw.Write(points[0, 0] + "," + points[0, 1]); stw.Write(" to ");
        stw.Write(points[2, 0] + "," + points[2, 1]); stw.Write(" to ");
        stw.Write(points[3, 0] + "," + points[3, 1]); stw.Write(" to ");
        stw.Write(points[0, 0] + "," + points[0, 1]);

        string ColorString;
        ColorString = "\"black\"";        
        
        stw.Write(" fs empty border lc rgb "+ColorString);
        stw.WriteLine();

        stw.Write("set label "); stw.Write("\"1\" font \",14\""); stw.Write(" at ");
        stw.Write(0.5 * (points[0, 0] + points[1, 0]) + "," + points[2, 1]);
        if (slope > 0) { stw.Write(" offset character -0.5,0.5"); } else { stw.Write(" offset character -0.5,-0.75"); };
        stw.WriteLine();

        stw.Write("set label "); stw.Write("\"" + slope.ToString("N1") + "\" font \",14\""); stw.Write(" at ");
        stw.Write(points[0, 0] + "," + 0.5 * (points[0, 1] + points[3, 1])); stw.Write(" right offset character -0.25,-0.25");

        gp.Cmd(stw.ToString());
    }

    gp.PlotLogXLogY(xSim, ySim, "Simulation", new PlotFormat(null, LineColors.Black, DashTypes.Dashed, 2, Styles.LinesPoints, PointTypes.OpenCircle));

    gp.Cmd($"set title \"Raster : {raster[i]}" + @"$\\, \\\frac{1}{cm}$" + "\" font \", 32\"");
    gp.Cmd("set xtics font \", 16\"");
    gp.Cmd("set ytics font \", 16\"");
    // gp.Cmd("set y2tics autofreq font \", 16\" textcolor \"red\"");
    // gp.Cmd("set y2tics autofreq font \", 16\" textcolor \"red\"");
    gp.Cmd("set bmargin 4");
    gp.Cmd("set key top left font \",24\"");
    gp.Cmd("set ylabel \"" + @"$\\\left.\\\frac{\\partial p}{\\partial x}\\right|_{stag} \\, \\\left[\\\frac{Pa}{m}\\right]$" + "\" font \",24\"");
    gp.Cmd("set xlabel \" " + @"$u_W \\, \\\left[\\\frac{m}{s}\\right] $" + "\" font \",24\"");
    gp.Cmd($"set border 1+2+4 lt 8");
    gp.Cmd($"set arrow 1 from 1.5, {0.9 * ySim.Min()} to 1.5, {1.1 * ySim.Max()} nohead lt 7");
}

In [ ]:
foreach(var plt in Plots){
    display(plt.PlotSVG(1200, 900));
}

In [ ]:
if(GenerateOutput){
    int i = 0;
    foreach(var plt in Plots){
        plt.PlotCairolatex().SaveTo("./PrintingNip/Figures/Part3/SimExp" + raster[i++] + ".tex");
    }
}

In [ ]:
double[] raster = new double[]{60.0, 70.0, 80.0, 100.0};
var gp = new Gnuplot();
gp.SetMultiplot(2, 2);
for(int i = 0; i < raster.Length; i++){
    int l = i / 2;
    int k = i % 2;

    Console.WriteLine("{0},{1}",l,k);

    gp.SetSubPlot(l,k);

    // data for experiment
    var Exp = ExpResult.Where(r => r.Item1 == raster[i]);
    var xExp = Exp.Select(r => r.Item2);
    var yExp = Exp.Select(r => r.Item3);
    var yDevExp = Exp.Select(r => r.Item4);

    // data for simulation
    var tab2plt = tab.ExtractRows((j,row) => Convert.ToDouble(row["id:delta"]) == 1e-5 && Convert.ToDouble(row["id:Raster"]).ApproxEqual(raster[i])); // arbitrary nip width
    var xSim = tab2plt.Select().Select(row => Convert.ToDouble(row["id:V_Wall"])).ToArray();
    var ySim = tab2plt.Select().Select(row => Convert.ToDouble(row["dPdXatStagnationPoint"])).ToArray();
    Array.Sort(xSim, ySim);
    // Add Data to plot
    gp.PlotXY(xExp, yExp, "Experiment", new PlotFormat(null, LineColors.Red, DashTypes.Solid, 2, Styles.LinesPoints, PointTypes.Circle), logX: true, logY: true);
    gp.PlotXY(xExp, yExp.Select((x, j) => x + yDevExp.ElementAt(j)), null, new PlotFormat(null, LineColors.Red, DashTypes.Dashed, 1, Styles.LinesPoints, PointTypes.Plus));
    gp.PlotXY(xExp, yExp.Select((x, j) => x - yDevExp.ElementAt(j)), null, new PlotFormat(null, LineColors.Red, DashTypes.Dashed, 1, Styles.LinesPoints, PointTypes.Plus));

    Plot2Ddata dat = new Plot2Ddata(xSim.Select(x => Math.Log10(x)), ySim.Select(x => Math.Log10(x)));
    double slope = Math.Round(dat.Regression().Average(kv => kv.Value) / 0.1) * 0.1;    
    // gp.PlotXY(new double[] {xSim.Min(), xSim.Max()}, new double[] {ySim.Average() * Math.Pow((xSim.Min()/xSim.Average()), slope), ySim.Average() * Math.Pow((xSim.Max()/xSim.Average()), slope) }, null, new PlotFormat(null, LineColors.Black, DashTypes.Solid, 1));

    // using (StringWriter stw = new StringWriter()) {
    //     double[,] points = new double[4, 2];
    //     points[0, 0] = xSim.Average();
    //     points[1, 0] = 1.2 * xSim.Average();
    //     points[2, 0] = 1.2 * xSim.Average();
    //     points[3, 0] = xSim.Average();

    //     points[0, 1] = ySim.Average();
    //     points[1, 1] = ySim.Average();
    //     points[2, 1] = ySim.Average() * Math.Pow(1.2, slope);
    //     points[3, 1] = ySim.Average() * Math.Pow(1.2, slope);

    //     stw.Write($"set object {i+1} poly from ");
        
    //     stw.Write(points[0, 0] + "," + points[0, 1]); stw.Write(" to ");
    //     stw.Write(points[2, 0] + "," + points[2, 1]); stw.Write(" to ");
    //     stw.Write(points[3, 0] + "," + points[3, 1]); stw.Write(" to ");
    //     stw.Write(points[0, 0] + "," + points[0, 1]);

    //     string ColorString;
    //     ColorString = "\"black\"";        
        
    //     stw.Write(" fs empty border lc rgb "+ColorString);
    //     stw.WriteLine();

    //     stw.Write("set label "); stw.Write("\"" + slope.ToString("N1") + "\" font \",14\""); stw.Write(" at ");
    //     stw.Write(0.5 * (points[0, 0] + points[1, 0]) + "," + points[2, 1]);
    //     if (slope > 0) { stw.Write(" offset character -0.5,0.5"); } else { stw.Write(" offset character -0.5,-0.75"); };

    //     gp.Cmd(stw.ToString());
    // }

    gp.PlotXY(xSim, ySim, "Simulation", new PlotFormat(null, LineColors.Black, DashTypes.Dashed, 2, Styles.LinesPoints, PointTypes.OpenCircle));
    

    if(i != 0){
        gp.Cmd("unset key");
    }else{
        gp.Cmd("set lmargin 8");
        gp.Cmd("set bmargin 3");
        gp.SetYRange(1e5, 1e7);
        gp.SetXRange(0.1, 2);
        gp.Cmd("set key top left font \",24\"");
        gp.Cmd("set key width -4");

    }
    if(k != 0){
        gp.Cmd("set ytics font \", 16\" nomirror");

        gp.Cmd("set format y \"\"");
        gp.Cmd("unset ylabel");
    } else {
        gp.Cmd("set format y \"10^%T\"");
        gp.Cmd("set ytics font \", 16\" nomirror");
        gp.Cmd("set ylabel \"" + @"$\\\left.\\\frac{\\partial p}{\\partial x}\\right|_{stag} \\, \\\left[\\\frac{Pa}{m}\\right]$" + "\" font \",24\"");
    }
    if(l != 1){
        gp.Cmd("set xtics font \", 16\" nomirror");

        gp.Cmd("set format x \"\"");
        gp.Cmd("unset xlabel");
    } else {
        gp.Cmd("set format x \"%.1f\"");
        gp.Cmd("set xtics font \", 16\" nomirror");
        gp.Cmd("set xlabel \" " + @"$u_W \\, \\\left[\\\frac{m}{s}\\right] $" + "\" font \",24\"");
    }

    gp.Cmd($"set title \"Raster : {raster[i]}" + @"$\\, \\\frac{1}{cm}$" + "\" font \", 32\"");

    gp.WriteDeferredPlotCommands();
    gp.Cmd($"unset object {i+1}");
    gp.Cmd($"unset label");

}
gp.UnsetMultiplot();

In [ ]:
gp.GetAllCommandsString()

In [ ]:
gp.PlotSVG(1200, 900)

In [ ]:
if(GenerateOutput){
    gp.PlotCairolatex(xSize: 20, ySize: 15).SaveTo("./PrintingNip/Figures/Part3/SimExpAll.tex");
}   

### Test plotting

In [ ]:
// Test code - see TestPDF.tex for output preview
// int i = 1;
// var gp = new Gnuplot();
// gp.PlotLogSlope(plts.Pick(1), format: new PlotFormat(lineColor: (LineColors)7));
// gp.Cmd("set key box width -5");
// plts.Pick(i).ToGnuplot(gp);
// gp.PlotCairolatex().SaveTo("./Figures/Test/Part3.tex");